In [4]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: John, Gen, Timothy
### Team Strategy Chosen: Risky 

Requirements for this assignment:
- Code needs to be dynamic
- Must read in a csv file containing a finite number of stock tickers (with an example csv file in this repo)
- Portfolio must contain only US listed stocks
- Must have an average daily volume of at least 10 000 shares as calculated based on the time interval July 2, 2021 to October 22, 2021
- Pick a minimum 10 maximum 20 stocks for portfolio.
- If we choose n stocks, each stock must make up minimum (100/(2n))% of the portfolio when weighted by value as of closing prices on November 26, 2021
- No individual stock may make up more than 35% of the portfolio when weighted by value
- We have $100000 USD to spend on portfolio and all must be spent
- When code is run with the .csv file, it must create a DataFrame "FinalPortfolio"
    - With this portfolio, index starts at 1 and ends at the number of stocks that our code chooses.
    - Headings must be: Ticker, Price, Shares, Value, Weight
        - Ticker is the ticker selected
        - Price is the price on November 26, 2021
        - Shares is the number of shares purchased (can be fractional)
        - Value is the total value of those shares
        - Weighted is the weight that the value of shares represents relative to the value of the portfolio (which is $100,000)
    - Needs to show that the total adds up to $100,000
    - Also need to show the weights add to 100%
    - This DataFrame must be printed to the screen as the second to last output to the screen.
- After the DataFrame, one final DataFrame called "Stocks" which has the same index as "Final Portfolio" must be mad
    - Only has the Tickers and Shares from "Final Portfolio"
    - Must output this DataFrame to a csv file titled "Stocks_Group_XX.csv"

In [5]:
stocks_from_csv = pd.read_csv('Tickers.csv')
stocks_from_csv


,AAPL
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG
...,...
56,TXN
57,UNH
58,UNP
59,UPS


In [6]:
stocks_lst = []
for g in range(len(stocks_from_csv.iloc[:,0])):
   stocks_lst.append(stocks_from_csv.iloc[g,0])
stocks_lst.append(stocks_from_csv.columns[0]) 
stocks_lst

['ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NKE',
 'ORCL',
 'OXY',
 'PCLN',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'SHOP.TO',
 'SBUX',
 'SLB',
 'SO',
 'SPG',
 'T',
 'TD.TO',
 'TGT',
 'TWX',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'AAPL']

In [7]:
len(stocks_from_csv.iloc[:,0])

61

In [9]:
len(stocks_lst)

62

In [10]:
stocks_from_csv.columns[0]

'AAPL'

In [23]:
from multiprocessing.pool import ThreadPool as Pool

pool_size = len(stocks_lst)

us_stock_lst = []
def filter_us():
    i=0
    while i < len(stocks_lst):
        if (yf.Ticker(stocks_lst[i]).info['regularMarketPrice'] != None) and (yf.Ticker(stocks_lst[i]).info['market']=='us_market'):
            us_stock_lst.append(yf.Ticker(stocks_lst[i]))
        i+=1

pool = Pool(pool_size)

for g in range(len(stocks_lst)):
    pool.apply_async(filter_us,)

pool.close()
pool.join()

KeyboardInterrupt: 

In [21]:
us_stock_lst

[]

In [22]:
len(us_stock_lst)

0

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here. 